In [1]:
import pandas as pd
import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
skr_music_file_paths = glob2.glob('DailyTopSongsSouthKorea/*.csv')

In [3]:
# skr_music_file_paths

In [4]:
#Add the dates to the files - run once
def addsDatesToData():
    to_remove_front = "DailyTopSongsSouthKorea\\regional-kr-daily-"
    to_remove_back = ".csv"

    len_front_remove = len(to_remove_front)
    len_back_remove = len(to_remove_back)

    for file in skr_music_file_paths: 
        the_data_in_file = pd.read_csv(file)        
        
        the_dates = file[len_front_remove:]
        the_dates = the_dates[:-len_back_remove]
        start_date = the_dates

        len_dates = the_data_in_file.shape[0]
        start_dates = len_dates * [start_date]
        
        date_columns = {"Start_Date": start_dates}   

        the_data_in_file["Start_Date"] = start_dates

        #write the data back to the File
        the_data_in_file.to_csv(file, index=False)
        
#remove dates in existing files
def removeExistingDates():
    for file in global_music_file_paths:
        
        the_data_in_file = pd.read_csv(file)

        the_data_in_file.drop(["Start_Date"], axis=1, inplace=True) 

        the_data_in_file.to_csv(file, index=False)

In [5]:
# removeExistingDates()
addsDatesToData()

In [6]:
#Combine the files and sort by dates
all_files = []
trouble_files = []
for file in skr_music_file_paths:
    the_data_in_file = pd.read_csv(file)
    
    if("URL" in list(the_data_in_file.columns)):
    
        the_data_in_file = the_data_in_file.drop(["URL"], axis=1)
    
    list_vals = sum(list(the_data_in_file.isnull().sum()))
    
    if(list_vals > 0):
        trouble_files.append(file)
    all_files.append(the_data_in_file)
    
the_spotify_data = pd.concat(all_files) 

In [7]:
the_spotify_data.drop(["Start Date"],axis=1, inplace=True)

In [8]:
the_spotify_data.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'days_on_chart', 'streams', 'Start_Date'],
      dtype='object')

In [9]:
the_spotify_data["Week"] = the_spotify_data["Start_Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["Year"] = the_spotify_data["Week"]

the_spotify_data['Week'] = the_spotify_data['Week'].apply(lambda x : x.isocalendar()[1] )
the_spotify_data['Year'] = the_spotify_data['Year'].apply(lambda x : x.isocalendar()[0] )

the_spotify_data["Start_Date_Dt"] = the_spotify_data["Start_Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [10]:
the_spotify_data["Artist"] = the_spotify_data["artist_names"].apply(lambda x : x.split(",")[0])

In [11]:
the_spotify_data.shape

(10976, 14)

In [12]:
#find the number of artists on the track
the_spotify_data.dropna(inplace=True)

the_spotify_data["Artist"] = the_spotify_data["Artist"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["Artist"] = the_spotify_data["Artist"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["Artist"] = the_spotify_data.apply(lambda x : x["Artist"] + "Artist_NA" if(x["Artist"] == "") else x["Artist"], axis=1)

the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["track_name"] = the_spotify_data.apply(lambda x : x["Artist"] + "_NA" if(x["track_name"] == "") else x["track_name"], axis=1)

def countNumberArtists(x):
    len_all_featured_artists = 0
    num_artists = 0
    xvals = x.split("feat")
    len_xvals = len(xvals)
    
    if(len_xvals > 1):
        featured_artists = xvals[1:]
        featured_artists_str = ' '.join(featured_artists)
        all_featured_artists = featured_artists_str.split("& ")
        len_all_featured_artists = len(all_featured_artists)
        
    num_artists = 1 + len_all_featured_artists
    
    return num_artists
        
    
the_spotify_data['ArtistCount'] = the_spotify_data['track_name'].apply(lambda x : countNumberArtists(x))

In [13]:
the_spotify_data.shape

(10976, 15)

In [14]:
# def everInTopTen(x):
#     position_vals = list(x["previous_rank"])
#     position_vals = list(set(position_vals))
    
# #     print(position_vals)
    
#     min_pos = min(position_vals)
#     existence_val = 0
    
#     if(min_pos <= 10):
#         existence_val = 1
    
#     x["isTopTen"] = [existence_val] * len(x)
    
#     return x
    
# the_spotify_data = the_spotify_data[~the_spotify_data.index.duplicated()]
# the_spotify_data = the_spotify_data.groupby(["Artist", "track_name"]).apply(lambda x : everInTopTen(x))
# the_spotify_data.drop(["Artist", "track_name"], inplace=True, axis=1)
# the_spotify_data = the_spotify_data.reset_index()
# the_spotify_data.drop(["level_2"], inplace=True, axis=1)

In [15]:
# the_spotify_data.shape

### how long does a song last on the charts?

In [17]:
def lengthSong(x):
    min_date = min(x["Start_Date"])
    max_date = max(x["Start_Date"])
    
    def convertStrToDate(y):
        split_date = y.split("-")
        dateindtformat = datetime(int(split_date[0]), int(split_date[1]), int(split_date[2]))
        
        return dateindtformat
    
    x["lenOnCharts"] = [int(((convertStrToDate(max_date) - convertStrToDate(min_date)).total_seconds() / 604800)) ] * len(x)
    x["lenOnCharts"] = x["lenOnCharts"]
    
    return x
    
the_spotify_data = the_spotify_data.groupby(["Artist", "track_name"], as_index=False).apply(lambda x : lengthSong(x))

C:\Users\lkhum\AppData\Local\Temp\ipykernel_44244\2557894937.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data = the_spotify_data.groupby(["Artist", "track_name"], as_index=False).apply(lambda x : lengthSong(x))


In [18]:
# the_spotify_data["isTopTen"].value_counts()

In [19]:
the_spotify_data.shape

(10976, 16)

In [20]:
unique_artists = the_spotify_data["Artist"].unique().tolist()
num_artists = len(unique_artists)
print(num_artists)

198


In [21]:
def getArtistAppearanceCount(x):
    
    x = x.reset_index()
    # x.drop(["index"],axis=1,inplace=True)
    # x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('level_1', 'artistAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x        
    
# the_spotify_data =the_spotify_data.groupby(["Artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))
# the_spotify_data.drop(["level_0"], inplace=True, axis=1)

In [22]:
def getTrackAppearanceCount(x):
    
    x = x.reset_index()   
    x_cols = x.columns
    x_cols = [w.replace('level_1', 'artistAppearanceCount') for w in x_cols]
    x.columns = x_cols    
    return x         
    
# the_spotify_data =the_spotify_data.groupby(["Artist", "track_name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))
# the_spotify_data.drop(["level_0"], inplace=True, axis=1)

In [23]:
the_spotify_data["peak_rank"] = the_spotify_data["peak_rank"].astype(int)
the_spotify_data["previous_rank"] = the_spotify_data["previous_rank"].astype(int)
the_spotify_data["streams"] = the_spotify_data["streams"].astype(int)
the_spotify_data["ArtistCount"] = the_spotify_data["ArtistCount"].astype(int)

# the_spotify_data["isTopTen"] = the_spotify_data["isTopTen"].astype(int)
the_spotify_data["lenOnCharts"] = the_spotify_data["lenOnCharts"].astype(int)
the_spotify_data["Year"] = the_spotify_data["Year"].astype(int)
the_spotify_data["Week"] = the_spotify_data["Week"].astype(int)

In [24]:
def trackAppearance(x):
#     x = x.reset_index()

    x = x.sort_values(by=["Start_Date"])
#     x.set_index(["Position"])

    x["previous_rank"] = x["previous_rank"].astype(int)
#     print(list(x["Position"]))
    
    x["Position_Difference"] = x["previous_rank"].diff().fillna(0)
    x["Position_Difference"] = x["Position_Difference"].astype(int)
    
#     x["Position Difference"] = x["Position"].sub(x["Position"].shift())
#     x.drop(["index"], inplace=True)
    
    return x
    
all_files_grouped = the_spotify_data.groupby(["Artist", "track_name"]).apply(lambda x: trackAppearance(x))

C:\Users\lkhum\AppData\Local\Temp\ipykernel_44244\1346119107.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_files_grouped = the_spotify_data.groupby(["Artist", "track_name"]).apply(lambda x: trackAppearance(x))


In [25]:
def positionvertime(x):
    posovertime = 0
    #new track
    if(x["Position_Difference"] == 0 and x["trackAppearanceCount"]==0):
        posovertime = 0
    #track stayed in the same position
    elif(x["Position_Difference"] == 0 and x["trackAppearanceCount"]!=0):
        posovertime = 50
    #track went up the chart
    elif(x["Position_Difference"] < 0):
        posovertime = 75
    #track fell down the chart
    else:
        posovertime = 100
    return posovertime

In [26]:
# all_files_grouped["PositionoverTime"] = all_files_grouped.apply(lambda x : positionvertime(x), axis=1)

In [27]:
all_files_grouped.to_csv("Classification/DailyTop200/all_files_KR_DailyTop200.csv", index=False)